I am a data scientist, who knows how to communicate with the network operator of Toronto cars. so I'd like to perform the following operations:

Use Case 1. Connect with the Toronto cars network, and send a message to all cars on that network. 

Use Case 2. Connect with the Toronto cars network, and send a query to all cars asking them about a certain dataset availability, and request an open channel with the cars that confirm having my dataset request and their interest in interacting with my session.

Some basic concepts to understand:

1. An address is spinned dynamically to represent the final destination of my message. Hops in between are going to forward my message to the relevant servers. 
2. The first point of contact that I'm aware of is going to be the Location that I will send my message to. eg. in this case I send my message to the Toronto Network, but I inform the network through the address field that I'm interested in contacting all cars under the Toronto Network. The address in this case, represents this desire and the final destination.
3. In this case, the Toronto Network is a "known location" to me, because I know how to contact it.
4. The method by which I contact the Toronto network is called "Route", a route embeds a connection to the location I'm trying to contact.

In order for this notebook to work we need to play a few roles: 
1. The data scientist who knows "Toronto Network"
2. The Toronto Network itself, which knows how to contact Cars in its known locations.
3. A car that's subscribed to the Toronto Network that will receive the message and reply back.


First step, let's setup some broadcasting utility functions.

In [1]:
import paho.mqtt.client as paho
import syft as sy


In [2]:
class PahoClient():

    def __init__(self, host="localhost", port=1883):
        self.host = host
        self.port = port
        self.client = paho.Client()
        self.client.on_subscribe = self.on_subscribe
        self.client.on_message = self.on_message
        self.client.on_publish = self.on_publish
        self.client.on_log = self.on_log

    def on_log(self, client, obj, level, string):
        print(string)

    def on_publish(self, client, userdata, mid):
        pass
    
    def on_subscribe(self, client, userdata, mid, granted_qos):
        print("Subscribed: "+str(mid)+" "+str(granted_qos))

    def on_message(self, client, userdata, msg):
        print(msg.topic+" "+str(msg.qos)+" "+str(len(msg.payload)))        
        
        # TODO automatically resolve message type.
        return self.recv_msg_without_reply(msg = msg.payload)

    def send(self, topic, msg):
        (rc, mid) = self.client.publish(topic, str(msg), qos=1, retain=False)

    def connect(self):
        self.client.connect(self.host, self.port)

    def listen(self, topic, background = True):
        if not self.client.is_connected():
            self.connect()
        self.client.subscribe(topic, qos=1)
        if not background:
            self.client.loop_forever()
        else:
            self.client.loop_start()
            
    def disconnect(self):
        self.client.loop_stop() 

In [3]:
class BroadcastClientConnection(PahoClient, sy.core.io.connection.ClientConnection): 
    def send_msg_with_reply(self, msg): 
        if not self.client.is_connected(): 
            self.connect() 
        self.send(self.topic, msg) 
    def send_msg_without_reply(self, msg): 
        if not self.client.is_connected(): 
            self.connect() 
        self.send(self.topic, msg) 

In [4]:
class BroadcastServerConnection(PahoClient, sy.core.io.connection.ServerConnection):
    def __init__(self, node):
        self.node = node
        super().__init__()

    def recv_msg_with_reply(self, msg):
        return self.node.recv_msg_with_reply(msg=msg)

    def recv_msg_without_reply(self, msg):
        self.node.recv_msg_without_reply(msg=msg)


In [5]:
# Help the address tell that we have no preference in who picks up the call.
class Unspecified(object):
    def __repr__(self):
        return "unspecified"

# Help the address tell that we're interested in all nodes.
class All(object):
    def __repr__(self):
        return "all"


## Step 1: as a data scientist who knows the network, let's send the network a message.

In [6]:
# I know the location of this network through this topic.
_ = sy.core.io.location.Location()
toronto_cars_network = sy.core.io.location.SubscriptionBackedLocationGroup(topic = 'toronto-cars', known_group_members = [_])

In [7]:
# let's self identify (future: this will be done through auth)
iam = sy.common.object.ObjectWithID()

# setup a route to this known location
route = sy.core.io.route.BroadcastRoute(source = iam.id, destination = toronto_cars_network, connection = BroadcastClientConnection())

# cotact toronto cars network.
network_client = sy.Network(name = "toronto-cars").get_client(routes = [route])

# now let's create a message that has an address
address = sy.core.io.address.address(network = network_client.name, domain = All(), device = Unspecified(), vm = Unspecified())

# now let's send them a message
network_client.send_msg_without_reply(msg = sy.ReprMessage(address = address))

AttributeError: 'ObjectStore' object has no attribute 'protobuf_type'

### Great! now that we know we can send a message to the network, let's pretend to be the network and see what happens on the network's end.

first we have to start a network server that can listen on messages.

In [ ]:
# start a network server to listen on messages.
network_node = sy.Network(name = "toronto-cars")
network_operator_server = BroadcastServerConnection(network_node)

In [ ]:
network_operator_server.listen(topic = 'toronto-cars')

now that someone is listening, let's send the message again.

In [ ]:
network_client.send_msg_without_reply(msg = sy.ReprMessage(address = address))

In [ ]:
network_operator_server.disconnect()

Awesome! now we know the network got the message. The exception you see above is because of trying to send a python object over the wire
we have to introduce serialization to make this problem disappear.

### STEP 2: Now let's pretend to be a domain in the Toronto Cars network and see if the network is going to forward me the message.


In [ ]:
domain_node = sy.Domain(name = 'mama-johns-car')

In [ ]:
domain_operator_server = BroadcastServerConnection(domain_node)

now let's assume that the network and its cars communicate on their own private channel, let's call it: "toronto-internal-car-channel"

before we listen to this channel, we have to make sure the network operator knows about it.

In [ ]:
network_operator_server.

In [ ]:
#domain_operator_server.listen("toronto-internal-car-channel")